This notebook shows how you could split Fashion outfits file in order to have train and test datasets to experiment with your model.

**Disclaimer**: The process used here is alway simple than the one we used to generate the test files for the leaderboard, but you could expect that some tricks used here (like dealing with products as a set, or generating candidates based on entire product_id list) are also used on the leaderboard files.

This code takes about 40 minutes to run in a common notebook

In [1]:
import datetime
print(datetime.datetime.now())

2022-06-13 10:18:36.479434


In [2]:
import pandas as pd

full_outfits = pd.read_parquet("../data/manual_outfits.parquet")
full_outfits.head()

,products,outfit_id
0,"[15360881, 15379678, 15781925, 16204075, 16260...",0
1,"[13893589, 13893721, 15426616, 16035469, 17173...",1
2,"[13508028, 14161732, 16160567, 17484491, 17503...",2
3,"[16127776, 16756133, 17040752, 18203427, 18205...",3
4,"[14480467, 15487690, 17257765]",4


To generate more examples given an particular outfit, one way is shuffing the products, remember that outfits are sets of products. This step is here just to ilustrate this possibility.

In [3]:
from numpy.random import permutation

full_outfits["products_shuffled"] = full_outfits.apply(lambda row: permutation(row["products"]).tolist(), axis=1)
full_outfits.head()

,products,outfit_id,products_shuffled
0,"[15360881, 15379678, 15781925, 16204075, 16260...",0,"[15360881, 15781925, 16260894, 16204075, 15379..."
1,"[13893589, 13893721, 15426616, 16035469, 17173...",1,"[13893721, 15426616, 18218977, 16035469, 13893..."
2,"[13508028, 14161732, 16160567, 17484491, 17503...",2,"[13508028, 16160567, 17503108, 17484491, 14161..."
3,"[16127776, 16756133, 17040752, 18203427, 18205...",3,"[17040752, 18203427, 16127776, 16756133, 18205..."
4,"[14480467, 15487690, 17257765]",4,"[15487690, 14480467, 17257765]"


To generate a train example to use in your model, you could split a outfit in incomplete_outfit and missing_product.

In [4]:
full_outfits["incomplete_outfit"] = full_outfits.apply(lambda row: row["products_shuffled"][:-1], axis=1)
full_outfits["missing_product"] = full_outfits.apply(lambda row: row["products_shuffled"][-1], axis=1)
full_outfits.head()

,products,outfit_id,products_shuffled,incomplete_outfit,missing_product
0,"[15360881, 15379678, 15781925, 16204075, 16260...",0,"[15360881, 15781925, 16260894, 16204075, 15379...","[15360881, 15781925, 16260894, 16204075]",15379678
1,"[13893589, 13893721, 15426616, 16035469, 17173...",1,"[13893721, 15426616, 18218977, 16035469, 13893...","[13893721, 15426616, 18218977, 16035469, 13893...",17173075
2,"[13508028, 14161732, 16160567, 17484491, 17503...",2,"[13508028, 16160567, 17503108, 17484491, 14161...","[13508028, 16160567, 17503108, 17484491]",14161732
3,"[16127776, 16756133, 17040752, 18203427, 18205...",3,"[17040752, 18203427, 16127776, 16756133, 18205...","[17040752, 18203427, 16127776, 16756133]",18205465
4,"[14480467, 15487690, 17257765]",4,"[15487690, 14480467, 17257765]","[15487690, 14480467]",17257765


In [5]:
items_metadata = pd.read_parquet("../data/products.parquet")
items_metadata = items_metadata["product_id"]
items_metadata.head()

0    17073270
1    17674562
2    17678603
3    17179699
4    15907453
Name: product_id, dtype: int32

One way to generate candidates is to sample then from all product_id in the dataset.

In [6]:
from random import randint

def candidates(row, minc=8, maxc=40):
    n = randint(minc, maxc)
#     n = 3
    c = items_metadata.sample(n).unique().tolist()
    c.append(row["missing_product"])
    return list(set(c))

full_outfits["candidates"] = full_outfits.apply(lambda row: candidates(row), axis=1)
full_outfits.head()

,products,outfit_id,products_shuffled,incomplete_outfit,missing_product,candidates
0,"[15360881, 15379678, 15781925, 16204075, 16260...",0,"[15360881, 15781925, 16260894, 16204075, 15379...","[15360881, 15781925, 16260894, 16204075]",15379678,"[17506183, 16528013, 17069200, 17008661, 16660..."
1,"[13893589, 13893721, 15426616, 16035469, 17173...",1,"[13893721, 15426616, 18218977, 16035469, 13893...","[13893721, 15426616, 18218977, 16035469, 13893...",17173075,"[17196711, 16744456, 18073609, 17416697, 17975..."
2,"[13508028, 14161732, 16160567, 17484491, 17503...",2,"[13508028, 16160567, 17503108, 17484491, 14161...","[13508028, 16160567, 17503108, 17484491]",14161732,"[17785380, 16639794, 17202616, 16795448, 16536..."
3,"[16127776, 16756133, 17040752, 18203427, 18205...",3,"[17040752, 18203427, 16127776, 16756133, 18205...","[17040752, 18203427, 16127776, 16756133]",18205465,"[17144965, 16154247, 17584907, 18205465, 16204..."
4,"[14480467, 15487690, 17257765]",4,"[15487690, 14480467, 17257765]","[15487690, 14480467]",17257765,"[16154241, 16328712, 16455305, 16507914, 18294..."


In [7]:
full_outfits.loc[0, "candidates"]

[17506183,
 16528013,
 17069200,
 17008661,
 16660374,
 15410584,
 18151198,
 16837158,
 16571943,
 16571695,
 16515636,
 17507254,
 16627129,
 17041467,
 17677755,
 16411898,
 17983835,
 15379678,
 17606243,
 17665124,
 16836968,
 17185769,
 16046451,
 16591990,
 17423993,
 16413050,
 16876031]

A example of split on 80% for training and 20% for test

In [8]:
train = full_outfits.sample(frac=0.8)
train.head()

,products,outfit_id,products_shuffled,incomplete_outfit,missing_product,candidates
49626,"[14032740, 16009213, 16020902, 16922916, 17442...",49626,"[16009213, 16020902, 14032740, 16922916, 17442...","[16009213, 16020902, 14032740, 16922916]",17442244,"[17442244, 18174663, 17990216, 17011370, 17742..."
208666,"[16291997, 16292011, 16292847]",208666,"[16292847, 16291997, 16292011]","[16292847, 16291997]",16292011,"[17168653, 16750862, 16843026, 16659219, 16511..."
20108,"[14726664, 15260529, 16173160, 16342704]",20108,"[14726664, 16342704, 15260529, 16173160]","[14726664, 16342704, 15260529]",16173160,"[17803783, 17232014, 17351185, 17732631, 16702..."
142179,"[16115367, 16505458, 16701212, 17796408, 17797...",142179,"[17797752, 17797762, 16115367, 16701212, 16505...","[17797752, 17797762, 16115367, 16701212, 16505...",17796408,"[17634946, 17638281, 15975948, 17925140, 16873..."
108620,"[14219148, 14427090, 14819016, 16090881, 17189...",108620,"[14819016, 17189952, 14427090, 14219148, 16090...","[14819016, 17189952, 14427090, 14219148]",16090881,"[16090881, 16464140, 17109260, 16282901, 17146..."


In [9]:
train["products"] = train.apply(lambda row: list(map(int, row["products"])), axis=1)
list_train = train.loc[0, "products"]
print(type(list_train[0]))

<class 'int'>


In [10]:
test = full_outfits[~full_outfits.outfit_id.isin(set(train["outfit_id"].values.tolist()))]
test.head()

,products,outfit_id,products_shuffled,incomplete_outfit,missing_product,candidates
2,"[13508028, 14161732, 16160567, 17484491, 17503...",2,"[13508028, 16160567, 17503108, 17484491, 14161...","[13508028, 16160567, 17503108, 17484491]",14161732,"[17785380, 16639794, 17202616, 16795448, 16536..."
6,"[14036151, 15394584, 17008577, 17152035, 17570...",6,"[17152035, 17570878, 14036151, 15394584, 17008...","[17152035, 17570878, 14036151, 15394584]",17008577,"[18151808, 16855170, 16268292, 17440646, 16640..."
7,"[13990742, 14371345, 15516565, 16487493]",7,"[13990742, 15516565, 16487493, 14371345]","[13990742, 15516565, 16487493]",14371345,"[16235009, 17996746, 14576365, 14371345, 18031..."
14,"[15408535, 15487690, 15602037, 16163007, 17052...",14,"[17052573, 18072093, 15408535, 18071104, 16163...","[17052573, 18072093, 15408535, 18071104, 16163...",15487690,"[17538052, 16214546, 17794067, 16369427, 15491..."
15,"[14573543, 15444115, 15597985, 16798256, 16946...",15,"[16946883, 17200690, 15597985, 15444115, 16798...","[16946883, 17200690, 15597985, 15444115, 16798...",14573543,"[16681858, 18208517, 17985175, 17722012, 16167..."


In [11]:
test_input = test[["outfit_id", "incomplete_outfit", "candidates"]]
test_input.head()

,outfit_id,incomplete_outfit,candidates
2,2,"[13508028, 16160567, 17503108, 17484491]","[17785380, 16639794, 17202616, 16795448, 16536..."
6,6,"[17152035, 17570878, 14036151, 15394584]","[18151808, 16855170, 16268292, 17440646, 16640..."
7,7,"[13990742, 15516565, 16487493]","[16235009, 17996746, 14576365, 14371345, 18031..."
14,14,"[17052573, 18072093, 15408535, 18071104, 16163...","[17538052, 16214546, 17794067, 16369427, 15491..."
15,15,"[16946883, 17200690, 15597985, 15444115, 16798...","[16681858, 18208517, 17985175, 17722012, 16167..."


In [12]:
test_output = test[["outfit_id", "missing_product"]]
test_output.head()

,outfit_id,missing_product
2,2,14161732
6,6,17008577
7,7,14371345
14,14,15487690
15,15,14573543


In [13]:
import time

# unique_name = int(time.time())
unique_name = "testJ13"
train.to_parquet(f"../data/manual_outfits_train_{unique_name}.parquet")
test_input.to_parquet(f"../data/manual_outfits_testinput_{unique_name}.parquet")
test_output.to_csv(f"../data/manual_outfits_testoutput_{unique_name}.csv", index=False)

In [14]:
print(datetime.datetime.now())

2022-06-13 11:18:00.907143
